<a href="https://colab.research.google.com/github/lev4ek0/texts/blob/main/Task_1_students__.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

**Ссылка**, на источник текста

In [ ]:
DATA_URL = "http://az.lib.ru/n/nekrasow_n_a/text_1840_pevitza.shtml"

Устанавливаем библиотеки

In [ ]:
! pip install -q PyYaml==5.3.1
! pip install -q rnnmorph==0.4.0
! pip install -q nltk==3.2.5
! pip install 'h5py==2.10.0' -force-reinstall

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.5 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.
Looking in links: orce-reinstall


Создаём объект морфологического анализатора `RNNMorph`

In [ ]:
%tensorflow_version 1.x
import warnings
warnings.filterwarnings('ignore')

from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [ ]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [ ]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Некрасов Николай Алексеевич. Певица</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/n/nekrasow_n_a/>Некрасов Николай Алексеевич</a><br>\r\nПевица</h2>\r\n\r'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [ ]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Толстой Алексей Константинович. Семья вурдалака\n\n\n\nТолстой Алексей Константинович\r\nСемья вурдалака\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n['

С помощью библиотеки [NLTK](https://nltk.org/) разбиваем текст на предложения и токены.

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"A total of 911 'sentences'"

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [ ]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent] 
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10] #Сейчас видно, что токены типа "точка", "запятая" и тд пока присутствуют в предложениях. От них нужно избавиться
for k, v in enumerate(predictions):
  for j in v[::-1]:
    if not j.isalpha():
      predictions[k].remove(j)
len(predictions)

sentences: 100%|██████████| 911/911 [00:00<00:00, 156464.15it/s]


911

Проверьте себя. Должны получиться следующие значения:

*   Предложений: 577 (возможны расхождения в несколько предложений)
*   Токенов: примерно 8621 (возможны расхождения в некоторое количество токенов)

In [ ]:
predictions

In [ ]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens)

11685

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 100 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам. 

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75. 

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [ ]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
russian_stop_words = stopwords.words("russian")
fdist = FreqDist(non_uniq_tokens)
dict1 = dict(fdist)
sorted_dict = {}
sorted_keys = sorted(dict1, key=dict1.get, reverse=True)

for w in sorted_keys:
    sorted_dict[w] = dict1[w]

import collections
counter = 0
top100 = collections.Counter(sorted_dict).most_common(100)
for pair in top100:
  if pair[0] in russian_stop_words:
    counter += 1
print((100 - counter) / 100) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
0.54


Проверьте себя: должно получиться 0.49 (допустимо небольшое расхождение)

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 50 раз.

Проверьте себя: должно получиться значение 22 (возможно небольшое расхождение)


In [ ]:
counter2 = 0
for i in top100:
  if i[1] > 20:
    counter2 += 1
counter2

76